In [ ]:
import sys
import os 
import numpy as np
import time
from PIL import Image
import src.siamese as siam
from src.region_to_bbox import region_to_bbox
from src.parse_arguments import parse_arguments

def main():
    #avoid printing TF debugging information(only show error log)
    os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
    hp,evaluation,run,env,design=parse_arguments()
    #build TF graph in siamese once for all
    filename,siam_net_z,loss=make_siameseFC(env,design)
    
    #iterate through all videos of evaluation.dataset
    if evaluation.video == 'all':
        #the path of folder of all videos
        train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset)
        videos_list=[v for v in os.listdir(train_data_folder)]
        videos_list.sort()
        num_v=len(videos_list)
        for i in range(num_v):
            gt,frame_name_list,frame_sz,n_frames=_init_train_video(env,evaluation,videos_list[i])
            start_frame=evaluation.start_frame
            #not sure
            #gt_=gt[start_frame:,:]
            gt_=gt[start_frame:]
            frame_name_list_=frame_name_list[start_frame:]
            num_frames=np.size(frame_name_list_)
            
            for j in range(num_frames-1):
                pos_x,pos_y,target_w,target_h=region_to_bbox(gt_[j])
                #train_siam_net(design,hp,frame_name_list,z_index,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss)
                train_siam_net(design,hp,frame_name_list,j,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss)
        
    else:
        gt,frame_name_list,_,_ = _init_train_video(env,evaluation,evaluation.video)
        gt_=gt[start_frame:]
        frame_name_list_=frame_name_list[start_frame:]
        num_frames=np.size(frame_name_list_)
        for i in range(num_frames-1):
            pos_x,pos_y,target_w,target_h=region_to_bbox(gt[evaluation.start_frame])
            train_siam_net(design,hp,frame_name_list,i,pos_x,pos_y,target_w,target_h,filename,siam_net_z,loss)
            
    write_file_param(desgin,env,evaluation)
            
    
    

In [ ]:
def _init_train_video(env,evaluation,video):
     #-------------------------------------------------------------------------
    #function//init info of a train video sequence
    #-------------------------------------------------------------------------
    #the path of train_data folder
    train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset,video)
    #os.listdir():show the file list of the folder
    frame_name_list=[f for f in os.listdir(video_folder) if f.endwith("."+env.image_type)]
    frame_name_list=[os.path.join(train_data_folder,'')+s for s in frame_name_list]
    frame_name_list.sort()
    
    #get the info of first frame
    with Image.open(frame_name_list[0]) as img:
        frame_sz=np.asarray(img.size)
        ##????????????????????????????????????????????
        frame_sz[1],frame_sz[0]=frame_sz[0],frame_sz[1]
        
        #read the initialization from ground truth(init the template_z)
    gt_file=os.path.join(video_folder,evaluation.gt_name)
    gt=np.genfromtxt(gt_file,delimiter=evaluation.delimiter)
    num_frame=len(frame_name_list)
    assert n_frames == len(gt),'number of frame and number of gt should be the same'
    
    return gt,frame_name_list,frame_sz,n_frames
    

In [ ]:
def write_file_param(design,env,evaluation):
    _layers_num=design.layers_num
    
    #get the time stamp now
    time_s=time.strftime('%Y-%m-%d-%H:%M:%S',time.localtime(time.time()))
    re_filename=str(time_s)+evaluation.train_re_param_file
    re_file_path=os.path.join(env.train_result_param,re_filename)
    for i in range(_layers_num):
        scope_name='conv'+str(i+1)
        with tf.variable_scope(scope_name,ruese=True):
            W_=tf.get_variable("W",shape=[])
            b_=tf.get_variable("b",shape=[])
            w_name='W'+str[i]
            b_name='b'+str[i]
    With tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(_layers_num):
            w_name='W'+str[i]
            b_name='b'+str[i]
            graph_w=convert_variables_to_constants(sess,sess.graph_def,[w_name])
            tf.train.wirte_graph(graph_w,'.',re_file_path,as_text=False)
            graph_b=convert_variables_to_constants(sess,sess.graph_def,[b_name])
            tf.train.write_graph(graph_b,'.',re_file_path,as_text=False)
            
            
            
    